In [4]:
import pandas as pd

Загрузим табличку с нашими данными про длительность поездок такси! 

In [5]:
df = pd.read_csv('taxi_dataset.csv', index_col=0)

df.head()

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,,
id2875421,1,2016-03-14 17:24:55,930.399753,0,455.0,2.012516
id2377394,0,2016-06-12 00:43:35,930.399753,0,663.0,3.070207
id3858529,1,2016-01-19 11:35:24,930.399753,0,2124.0,5.541026
id3504673,1,2016-04-06 19:32:31,930.399753,0,429.0,2.843521
id2181028,1,2016-03-26 13:30:55,930.399753,0,435.0,2.481962


Попробуем построить модель Линейной Регрессии!

В качестве Выборки используйте таргетную колонку (*trip_duration*) и все признаки, кроме времени начала заказа (*pickup_datetime*)

Для начала воспользуемся "коробочным решением". Для этого создайте переменную model, положите в нее класс **LinearRegression** из модуля **linear_model** библиотеки **sklearn**.

Далее воспользуйтесь методом **fit**, положите в него через запятую: массив **X**, состоящий из признаков объектов (можно pandas dataframe'ом, а можно numpy-массивом), и массив **Y** с таргетами.

In [6]:
from sklearn.linear_model import LinearRegression

In [7]:
X = df.drop(columns=["trip_duration", "pickup_datetime"])
y = df["trip_duration"]

In [9]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

Чтобы посмотреть на значения полученных коэффициентов модели, следует обратиться к атрибуту **coef_** класса линейной регрессии. Для просмотра свободного веса, к атрибуту **intercept_** 

In [17]:
coef = model.coef_
intercept = model.intercept_

In [18]:
print(f"The coefficients of the model are {', '.join([str(round(i, 3)) for i in coef])}")
print(f"The intercept of the model is {round(intercept, 3)}")

The coefficients of the model are 201.484, 0.393, 147.387, 44.623
The intercept of the model is 301.624


Теперь реализуйте функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуйте решение через аналитическое решение в матрицах из лекции и практики:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [19]:
import numpy as np


def LinearRegressionByMatrix(X: np.array, Y: np.array, fit_intercept: bool = True):
    """
    :param X: матрица объектов
    :param Y: вектор (матрица с 1 столбцом) ответов
    :param fit_intercept: добавить ли константную переменную в данные?

    :return: одномерный numpy-array с полученными коэффициентами beta
    """

    X = np.c_[X, np.ones(X.shape[0])]
    return np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), Y)

In [20]:
X_train = df.drop(['trip_duration', 'pickup_datetime'], axis=1).values
Y_train = df['trip_duration'].values

LinearRegressionByMatrix(X_train, Y_train)

array([201.48355138,   0.39255178, 147.38701156,  44.62299718,
       301.62412993])

Получились ли коэффициенты такими же, как и в "коробочном" варианте?

Имея коэффициенты модели, можно восстановить предсказания для каждого объекта!

Сделайте это через операцию матричного произведения матрицы **X** и полученных коэффициентов (как результат работы *LinearRegressionByMatrix*)

In [22]:
betas = LinearRegressionByMatrix(X_train, Y_train)

np.dot(np.c_[X_train, np.ones(X_train.shape[0])], betas)

array([ 958.14228109,  803.85604957, 1115.5949386 , ..., 1226.45694736,
        700.4159849 ,  843.88702965])